In [1]:
import matplotlib.pyplot as plt

from PIL import Image
from scipy.misc import imread
from scipy.misc import imsave
import numpy as np
from scipy.stats import multivariate_normal
import math

In [2]:
## Read image
data_path = './Dataset/hw3_img.jpg'
img = Image.open(data_path) # Image object
image = np.array(img) # Array

# get shape
im_w = image.shape[0]
im_h = image.shape[1]

resize_ratio = 0.1
img_resize = img.resize((int(im_h*resize_ratio), int(im_w*resize_ratio))) # Image object
image_resize = np.array(img_resize) # Array

# get shape
im_w_re = image_resize.shape[0]
im_h_re = image_resize.shape[1]

## Normalize pixel value to 0 - 1
im = image/255.0

## Avoid sigular matrix of Cov_mat
bias = 0.0001


# print(im_w,im_h)
# print(im_w_re,im_h_re)
# print(image.shape)
# print(image_resize.shape)

In [3]:
# setting para.
K_cluster = 3
means = np.zeros((K_cluster,3))
for i in range(K_cluster):
    for j in range(3):
        means[i,j] = i
k_ary = np.zeros((im_w_re,im_h_re))

In [ ]:
# kmeans
# for k means
# means = np.zeros((1,3,2))
# means[0,:,1]=np.array([1.0, 1.0, 1.0])


def k_means(k_ary, image, means):    
    # init
    init_means = np.zeros_like(means)
    init_cov = np.zeros((K_cluster,3,3))
    init_pi = np.zeros(K_cluster)  
    
    # cal k-array
    for i in range(im_w_re):
        for j in range(im_h_re):
            dist = np.zeros(K_cluster)
            for k in range(K_cluster):
                dist[k] = np.sum((image[i,j,:]-means[k,:])**2)
            k_ary[i,j] = np.argmin(dist)  

    # start update means
    for k in range(K_cluster):
        k_cluster_list = np.where(k_ary == k)
        k_lst_L = len(k_cluster_list[0])
        
        # cal for pi
        init_pi[k] = float(k_lst_L)/float((im_w_re*im_h_re))
        
        # use for create mean 
        if k_lst_L != 0:
            R = image[:,:,0]
            G = image[:,:,1]
            B = image[:,:,2]
            init_means[k,0] = float(R[k_cluster_list].sum())/k_lst_L
            init_means[k,1] = float(G[k_cluster_list].sum())/k_lst_L
            init_means[k,2] = float(B[k_cluster_list].sum())/k_lst_L
            
        # cal Cov.
        N_k = k_lst_L
        val = init_cov[k,:,:]
        
        for i in range(im_w_re):
            for j in range(im_h_re):
                if k_ary[i,j] == k:
                    val += np.dot((image[i,j,:] - init_means[k,:]).reshape((3,1)), (image[i,j,:] - init_means[k,:]).reshape((1,3)))
        init_cov[k,:,:] = val/N_k

    return k_ary, init_means, init_cov, init_pi


IterN = 100
for i in range(IterN):
    k_ary, means, cov, pi = k_means(k_ary, image_resize, means)

print ('======================================')
print ('kmeans - means = \n',means)
print ('kmeans - cov = \n',cov)
print ('kmeans - pi = \n',pi)

# print(means[0,:])
# print(cov[0,:,:])

kmeans - means = 
 [[  49.89148391   33.17430931    8.83309598]
 [ 188.98232984  133.66492147   46.32918848]
 [ 207.42582106  200.84409211  175.25443564]]
kmeans - cov = 
 [[[ 1300.61568082   705.91267783   162.8773596 ]
  [  705.91267783   743.46890422   237.09944325]
  [  162.8773596    237.09944325   282.92799608]]

 [[ 2040.12730557   599.56083304  -433.94837167]
  [  599.56083304  2536.15801033   628.31809195]
  [ -433.94837167   628.31809195  1097.38312709]]

 [[ 1428.44381798   738.719088      39.26877939]
  [  738.719088     800.37395623   569.35865682]
  [   39.26877939   569.35865682  1344.23801826]]]
kmeans - pi = 
 [ 0.38096788  0.33159722  0.2874349 ]


In [ ]:
# for EM algorithm   
likelihood = []
iter_num = 100

def Cal_mean(image, means, gamma_mat):
    new_means = np.zeros((K_cluster,3))   
    
    for k in range(K_cluster):
        r = 0.0
        g = 0.0
        b = 0.0
        N_k = np.sum(gamma_mat[:,:,k])
        for i in range(im_w_re):
            for j in range(im_h_re):
                r += image[i,j,0]*gamma_mat[i,j,k]
                g += image[i,j,1]*gamma_mat[i,j,k]
                b += image[i,j,2]*gamma_mat[i,j,k]
        new_means[k,:] = np.array([r/N_k, g/N_k, b/N_k])
    return new_means

def Cal_cov(image, cov, means, gamma_mat):
    new_cov = np.zeros_like(cov)
    
    for k in range(K_cluster):
        val = new_cov[k,:,:]
        N_k = np.sum(gamma_mat[:,:,k])
        
        for i in range(im_w_re):
            for j in range(im_h_re):
                tmp = np.dot((image[i,j,:] - means[k,:]).reshape((3,1)), (image[i,j,:] - means[k,:]).reshape((1,3)))
                val += gamma_mat[i,j,k]*tmp
        new_cov[k,:,:] = val/N_k
    return new_cov

def Cal_pi(pis,gamma_mat):
    new_pis = np.zeros_like(pis)
    
    for k in range(K_cluster):
        N_k = np.sum(gamma_mat[:,:,k])
        new_pis[k] = N_k/float(im_w_re*im_h_re)
    return new_pis
    
def Cal_gamma(pixel, k, means, cov, pis):
    denom = 0.0
    gamma = 0
    nom = pis[k]*multivariate_normal.pdf(pixel, means[k,:], cov[k,:,:]+bias)
    for i in range(K_cluster):
        denom += pis[i]*multivariate_normal.pdf(pixel, means[i,:], cov[i,:,:]+bias)
    
    gamma = nom/denom
    return gamma

def EM_(image, means, covars, pis, k_array):
    
    # E step
    gamma_mat = np.zeros((im_w_re, im_h_re, K_cluster))
    for i in range(im_w_re):
        for j in range(im_h_re):     
            for k in range(K_cluster):
                gamma_mat[i,j,k] = Cal_gamma(image[i,j,:], k, means, covars, pis)
    
    # M step
    new_means = Cal_mean(image, means, gamma_mat)
    new_covars = Cal_cov(image, covars, new_means, gamma_mat)
    new_pis = Cal_pi(pis, gamma_mat)
    
    return gamma_mat,new_means, new_covars, new_pis

def log_likelihood(image, means, covars, pis):
    val = 0.0
    for i in range(im_w_re):
        for j in range(im_h_re):
            tmp = 0.0
            for k in range(K_cluster):
                # reshape mean to a vector
                mean_vec = means[k,:].reshape(-1)
                tmp += pis[k]*multivariate_normal.pdf(image[i,j,:], mean_vec, covars[k,:,:]+bias)
            val += np.log(tmp)
    return -val

print('Start EM Algorithm')
for i in range(iter_num):
    print('iter = ',i)
    
    gamma_mat, means, cov, pi = EM_(image_resize, means, cov, pi, k_ary)
    
    print ('EM - means = \n',means)
    print ('EM - cov = \n',cov)
    print ('EMs - pi = \n',pi)
    print ('=========================================')
    
    likelihood.append(log_likelihood(image_resize, means, cov, pi))
    


Start EM Algorithm
iter =  0
EM - means = 
 [[  48.7349642    34.19381591    7.90094627]
 [ 188.44336121  130.3626655    43.82213337]
 [ 203.93626597  196.86678551  172.24708683]]
EM - cov = 
 [[[ 1225.00186098   863.19435364   197.12296765]
  [  863.19435364   835.2546398    203.79729184]
  [  197.12296765   203.79729184   154.81288295]]

 [[ 2302.64391665   958.6161077   -331.8914007 ]
  [  958.6161077   3131.14652954   788.58487287]
  [ -331.8914007    788.58487287  1059.65368301]]

 [[ 1592.96841958   892.78102695   235.9011748 ]
  [  892.78102695   954.09786167   800.14246564]
  [  235.9011748    800.14246564  1498.93429437]]]
EMs - pi = 
 [ 0.37081146  0.32862506  0.30056348]
iter =  1
EM - means = 
 [[  47.38752695   34.27264573    6.97887565]
 [ 186.44261296  126.95995863   41.71778844]
 [ 201.65104451  194.31054346  169.61933702]]
EM - cov = 
 [[[ 1180.60981601   912.80334149   177.44806478]
  [  912.80334149   876.77584848   176.69545402]
  [  177.44806478   176.69545402   11

iter =  12
EM - means = 
 [[  31.52601925   21.5853644     2.03424897]
 [ 159.85590889  105.58796043   27.69921511]
 [ 193.72563472  183.98242336  153.82306343]]
EM - cov = 
 [[[  539.7485211    393.98146635    21.87303625]
  [  393.98146635   337.70564046    21.66041216]
  [   21.87303625    21.66041216    10.59940934]]

 [[ 4582.67904524  2620.96919491   357.93807994]
  [ 2620.96919491  3904.09747426   820.12842773]
  [  357.93807994   820.12842773   515.06005678]]

 [[ 2116.75694623  1489.4561778    944.62840691]
  [ 1489.4561778   1632.44111167  1632.05707018]
  [  944.62840691  1632.05707018  2634.5398758 ]]]
EMs - pi = 
 [ 0.24440397  0.37759138  0.37800465]
iter =  13
EM - means = 
 [[  30.04934852   20.31126617    1.7984954 ]
 [ 157.91900087  104.29795663   26.9714986 ]
 [ 193.38643377  183.53100896  153.12280794]]
EM - cov = 
 [[[  471.83015701   331.791938      13.19300002]
  [  331.791938     274.06544337    13.10272955]
  [   13.19300002    13.10272955     8.50915254]]

 [[

EM - means = 
 [[  27.47118063   18.51365081    1.04059351]
 [ 149.96720618   98.16217898   24.11335356]
 [ 191.89840775  181.28721906  149.49495999]]
EM - cov = 
 [[[  3.80251837e+02   2.74705067e+02   3.72197804e+00]
  [  2.74705067e+02   2.28647006e+02   3.66323163e+00]
  [  3.72197804e+00   3.66323163e+00   3.01750524e+00]]

 [[  5.26815599e+03   3.06230985e+03   4.86541475e+02]
  [  3.06230985e+03   4.03688223e+03   8.20766855e+02]
  [  4.86541475e+02   8.20766855e+02   4.37830535e+02]]

 [[  2.23214602e+03   1.61928726e+03   1.08744084e+03]
  [  1.61928726e+03   1.79137001e+03   1.81699579e+03]
  [  1.08744084e+03   1.81699579e+03   2.90783070e+03]]]
EMs - pi = 
 [ 0.20664091  0.39651923  0.39683986]
iter =  24
EM - means = 
 [[  27.41881039   18.47700986    1.01758165]
 [ 149.69433401   97.94957633   24.01408785]
 [ 191.8436315   181.19854008  149.35095868]]
EM - cov = 
 [[[  3.78496513e+02   2.73741226e+02   3.58484653e+00]
  [  2.73741226e+02   2.28087072e+02   3.53569085e+00]

EM - means = 
 [[  27.16999757   18.29366611    0.90666883]
 [ 148.3238017    96.88545109   23.50365286]
 [ 191.56822892  180.73771318  148.60512589]]
EM - cov = 
 [[[  3.70531282e+02   2.69270890e+02   2.91726530e+00]
  [  2.69270890e+02   2.25462736e+02   2.90523050e+00]
  [  2.91726530e+00   2.90523050e+00   2.34722894e+00]]

 [[  5.39254640e+03   3.14095938e+03   5.01839152e+02]
  [  3.14095938e+03   4.06795989e+03   8.19537008e+02]
  [  5.01839152e+02   8.19537008e+02   4.23350007e+02]]

 [[  2.25421231e+03   1.64288457e+03   1.11275088e+03]
  [  1.64288457e+03   1.82284455e+03   1.85325404e+03]
  [  1.11275088e+03   1.85325404e+03   2.96246392e+03]]]
EMs - pi = 
 [ 0.20108515  0.39810214  0.40081272]
iter =  35
EM - means = 
 [[  27.16046882   18.28627927    0.90238446]
 [ 148.26763838   96.84079365   23.4804986 ]
 [ 191.55498381  180.71514063  148.56878268]]
EM - cov = 
 [[[  3.70238584e+02   2.69102644e+02   2.89196537e+00]
  [  2.69102644e+02   2.25360680e+02   2.88024821e+00]

 [ 0.20006997  0.39810574  0.4018243 ]
iter =  45
EM - means = 
 [[  27.1167636    18.25204741    0.88259374]
 [ 148.00318236   96.62408713   23.36145464]
 [ 191.4804697   180.59004534  148.36879856]]
EM - cov = 
 [[[  3.68906494e+02   2.68334511e+02   2.77745836e+00]
  [  2.68334511e+02   2.24890687e+02   2.76526090e+00]
  [  2.77745836e+00   2.76526090e+00   2.23695361e+00]]

 [[  5.41721255e+03   3.15606243e+03   5.04212429e+02]
  [  3.15606243e+03   4.07463953e+03   8.18779736e+02]
  [  5.04212429e+02   8.18779736e+02   4.19653888e+02]]

 [[  2.26032141e+03   1.64927831e+03   1.11954709e+03]
  [  1.64927831e+03   1.83143287e+03   1.86301017e+03]
  [  1.11954709e+03   1.86301017e+03   2.97696156e+03]]]
EMs - pi = 
 [ 0.2000309   0.39809664  0.40187246]
iter =  46
EM - means = 
 [[  27.115109     18.25074107    0.8818387 ]
 [ 147.99290906   96.61527642   23.35623868]
 [ 191.47681688  180.58404555  148.35929515]]
EM - cov = 
 [[[  3.68856381e+02   2.68305565e+02   2.77318885e+00]
  [ 

iter =  56


In [ ]:
print(means)

In [ ]:
# reconstruct image
def construct_image(image, means, gamma_matrixs):
    k_num = gamma_matrixs.shape[2]
    new_image = np.zeros_like(image)
    for i in range(im_w_re):
        for j in range(im_h_re):
            compare_ls = []
            for k in range(k_num):
                compare_ls.append(gamma_matrixs[i,j,k])
            min_idx = np.argmax(compare_ls)
            new_image[i,j,0] = means[min_idx,0]
            new_image[i,j,1] = means[min_idx,1]
            new_image[i,j,2] = means[min_idx,2]
    return new_image

new_image = construct_image(image_resize, means, gamma_mat)    

# plot log likelihood & image
plt.figure()
plt.plot(range(len(likelihood)),likelihood, 'r')
plt.title('log likelihood')

plt.figure()
plt.imshow(image_resize)
plt.title('resize image')

plt.figure()
plt.imshow(new_image)
plt.title('construct image')

# save fig
plt.savefig('./K_%s_reconstructImg.png'%str(K_cluster))

plt.show()

In [ ]:
# print(new_pis)
#now [0.51673384  0.48326616]

In [ ]:
# print(np.array([[1,2,3],[1,2,3]]).sum())

In [ ]:
# export to python file
#!jupyter nbconvert --to python Problem3_Gaussian_Mixture_Model.ipynb